In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sparseSpACE
from sparseSpACE.Function import *
from sparseSpACE.StandardCombi import *
from sparseSpACE.Grid import *
import numpy as np
from numba import jit

## Clenshaw-Curtis Grid


In [ ]:
dim = 5
coeffs = np.ones(dim)*5
midpoint = np.ones(dim)*0.5
a = np.zeros(dim)
b = np.ones(dim)
grid = GaussLegendreGrid(a=a, b=b)
f = GenzDiscontinious(coeffs, midpoint)

# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = f.getAnalyticSolutionIntegral(a,b))
combiObject = StandardCombi(a, b, operation=operation)
minimum_level = 1
maximum_level = 5
combiObject.perform_operation(minimum_level, maximum_level, f)
#combiObject.print_resulting_combi_scheme()
#combiObject.print_resulting_sparsegrid()

## Gauss-Legendre Grid

In [ ]:
dim = 3
coeffs = np.ones(dim)
offset = 0.5
a = np.zeros(dim)
b = np.ones(dim)
grid = ClenshawCurtisGrid(a=a, b=b)
f = GenzOszillatory(coeffs=coeffs, offset=offset)
reference_solution = f.getAnalyticSolutionIntegral(a,b)

# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)
combiObject = StandardCombi(a, b, operation=operation, log_level=log_levels.WARNING)
minimum_level = 1
maximum_level = 5
combiObject.perform_operation(minimum_level, maximum_level, f)

## Spatially Adaptive Combination

In [ ]:
from sparseSpACE.spatiallyAdaptiveCell import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
#dimension of the problem
dim = 2
# define function to be integrated
coeffs = np.ones(dim)*5
#midpoint = np.ones(dim)*0.5
offset = 0.5
a = np.zeros(dim)
b = np.ones(dim)
grid = TrapezoidalGrid(a=a, b=b)
f = GenzOszillatory(coeffs, offset)
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# reference integral solution for calculating errors
reference_solution = f.getAnalyticSolutionIntegral(a,b)

# define error estimator for refinement
errorOperator=ErrorCalculatorSurplusCell()

# NEW! define operation which shall be performed in the combination technique
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = reference_solution, print_level=0)

# define Cell refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceCell = SpatiallyAdaptiveCellScheme(a, b, operation=operation)

# the cell method does not have component grids and therefore the sparse grid is printed twice per iteration!
# performing the spatially adaptive refinement with the Cell method
adaptiveCombiInstanceCell.performSpatiallyAdaptiv(2, 2, errorOperator, 10**-2, do_plot=False, print_output=False)

print("Number of points used in refinement:", adaptiveCombiInstanceCell.get_total_num_points())

In [ ]:
import sys
sys.path.append("..")

from src.py.option import sde_body, EuropeanOption, AsianOption


# Valuation Examples:  
## European Call Option

In [ ]:
def phi(x,sigma,mu,T,K, axis=1):
    payoffcoarse=np.exp(-mu * T)* np.maximum(x - K, 0.) 
    return payoffcoarse
eop = EuropeanOption()
dim = 1
eop_payout = lambda x: phi(eop.S_t(x), eop.sigma, eop.r, eop.T, eop.K)
sparse_payout = CustomFunction(eop_payout)

a = np.zeros(dim)
b = np.ones(dim)
grid = GaussLegendreGrid(a=a, b=b)
f = sparse_payout

# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=eop.scholes_call())
combiObject = StandardCombi(a, b, operation=operation)
minimum_level = 1
maximum_level = 7
combiObject.perform_operation(minimum_level, maximum_level, f)

## Asian Geometric Call Option

In [ ]:
aop = AsianOption(d=3)
dim = aop.d
aop_payout = lambda x: aop.payout_func(aop.S_t(x))
sparse_payout = CustomFunction(aop_payout)
a = np.zeros(dim)
b = np.ones(dim)
grid = ClenshawCurtisGrid(a=a+aop.epsilon, b=b-aop.epsilon)
f = sparse_payout

# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=aop.scholes_call())
combiObject = StandardCombi(a, b, operation=operation)
minimum_level = 1
maximum_level = 5
combiObject.perform_operation(minimum_level, maximum_level, f)

In [ ]:
# from scipy.stats import norm
# ar = np.asarray([0.11270167, 0.11270167, 0.11270167])
# #aop.payout_func(aop.S_t(ar))
# noise = np.random.uniform(0, 0.5, size=(int(1e4), 3))
# vec_noise = sparse_payout.eval_vectorized(noise)
# aop = AsianOption(d=8)
# aop.A[:-1, :-1]@norm.ppf(aop.t_v[:-1])
